In [8]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np

In [10]:
df = pd.read_parquet("iowa_liquor_2023_2025.parquet", columns=['store', 'store_location'])
df = df.dropna(subset=["store_location"]).drop_duplicates(subset=["store"])
df.head()

In [ ]:
from tqdm import tqdm

In [ ]:
df.store_location.iloc[1]

{'coordinates': array([-93.61378,  41.60575]), 'type': 'Point'}

In [ ]:
from math import radians

df['lon'] = df['store_location'].apply(
    lambda x: None if pd.isna(x) else x['coordinates'][0]
)
df['lat'] = df['store_location'].apply(
    lambda x: None if pd.isna(x) else x['coordinates'][1]
)
df["lat"] = df['lat'].astype(float).apply(radians)
df["lon"] = df['lon'].astype(float).apply(radians)
df.drop(columns=["store_location"], inplace=True)

In [ ]:
df

,invoice_line_no,date,store,name,address,city,zipcode,county,category,category_name,...,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons,lon,lat
0,INV-54554000001,2023-01-02,4829,CENTRAL CITY 2,1501 MICHIGAN AVE,DES MOINES,50314,POLK,1022200,100% AGAVE TEQUILA,...,12,750,14.50,21.75,12,261.00,9.00,2.37,-1.633869,0.726157
1,INV-54554000002,2023-01-02,4829,CENTRAL CITY 2,1501 MICHIGAN AVE,DES MOINES,50314,POLK,1031100,AMERICAN VODKAS,...,12,1000,4.65,6.98,60,418.80,60.00,15.85,-1.633869,0.726157
2,INV-54554000003,2023-01-02,4829,CENTRAL CITY 2,1501 MICHIGAN AVE,DES MOINES,50314,POLK,1032200,IMPORTED FLAVORED VODKA,...,12,750,9.96,14.94,24,358.56,18.00,4.75,-1.633869,0.726157
3,INV-54554000004,2023-01-02,4829,CENTRAL CITY 2,1501 MICHIGAN AVE,DES MOINES,50314,POLK,1081200,CREAM LIQUEURS,...,6,1000,17.00,25.50,12,306.00,12.00,3.17,-1.633869,0.726157
4,INV-54554000005,2023-01-02,4829,CENTRAL CITY 2,1501 MICHIGAN AVE,DES MOINES,50314,POLK,1062400,SPICED RUM,...,12,1000,12.49,18.74,60,1124.40,60.00,15.85,-1.633869,0.726157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6858345,RINV-05896000019,2025-08-29,6216,HY-VEE C-STORE #3 / CEDAR RAPIDS,2300 BOWLING GREEN STREET SOUTHWEST,CEDAR RAPIDS,52404,LINN,1031100,AMERICAN VODKAS,...,24,375,4.00,6.00,-1,-6.00,-0.37,-0.09,-1.599726,0.732262
6858346,RINV-05896100014,2025-08-29,5759,CASEY'S GENERAL STORE #42 / WELLMAN,700 6TH ST,WELLMAN,52356,WASHINGTON,1081600,WHISKEY LIQUEUR,...,8,100,6.00,9.00,-8,-72.00,-0.80,-0.21,-1.602848,0.723743
6858347,RINV-05896200016,2025-08-29,3879,TARGET STORE T-1800 / SIOUX CITY,5775 SUNNYBROOK DR,SIOUX CITY,51106,WOODBURY,1031200,AMERICAN FLAVORED VODKA,...,12,750,8.25,12.38,-12,-148.56,-9.00,-2.37,-1.681338,0.740851
6858348,RINV-05896300021,2025-08-29,010139,FAS MART #553 / COUNCIL BLUFFS,611 EAST BROADWAY,COUNCIL BLUFFS,51503,POTTAWATTAMIE,1081600,WHISKEY LIQUEUR,...,1,50,27.00,40.50,-1,-40.50,-0.05,-0.01,-1.672707,0.720240


In [ ]:
from sklearn.metrics.pairwise import haversine_distances
result = haversine_distances(df[["lat", "lon"]])

ValueError: Input contains NaN.

In [ ]:
pairwise_distance_miles = result * 3959

In [ ]:
np.fill_diagonal(pairwise_distance_miles, np.NaN)
pairwise_distance_miles

array([[         nan,   1.6838412 ,   2.06182724, ...,  14.49627859,
        109.58102776, 107.17792933],
       [  1.6838412 ,          nan,   3.6799311 , ...,  16.15968611,
        109.22127709, 106.82643472],
       [  2.06182724,   3.6799311 ,          nan, ...,  12.48839053,
        109.28296777, 106.87041282],
       ...,
       [ 14.49627859,  16.15968611,  12.48839053, ...,          nan,
        111.36677244, 108.91559646],
       [109.58102776, 109.22127709, 109.28296777, ..., 111.36677244,
                 nan,   2.45965014],
       [107.17792933, 106.82643472, 106.87041282, ..., 108.91559646,
          2.45965014,          nan]])

In [ ]:
df["# of stores within 5 mile radius"] = (pairwise_distance_miles < 5).sum(axis=1)


In [ ]:
df["Nearest other store (mi)"] = pd.DataFrame(pairwise_distance_miles).min(axis=1, skipna=True).to_numpy()


In [ ]:
df

,Store Number,Store Location,lon,lat,# of stores within 5 mile radius,Nearest other store (mi)
0,5245,POINT (-93.61379 41.57608),-1.633869,0.725639,86,0.566167
1,6303,POINT (-93.59738 41.55503),-1.633582,0.725272,64,0.051137
3,2190,POINT (-93.61979 41.60558),-1.633974,0.726154,98,0.024184
4,2699,POINT (-93.62362 41.70324),-1.634040,0.727859,63,0.140796
5,2636,POINT (-93.57411 41.60444),-1.633176,0.726134,80,0.298991
...,...,...,...,...,...,...
6835250,10653,POINT (-95.12921 43.38217),-1.660318,0.757162,21,0.007031
6847204,10639,POINT (-91.4257 41.48638),-1.595679,0.724074,1,0.258425
6847490,10652,POINT (-93.71378 41.77213),-1.635614,0.729061,6,0.018401
6853047,10657,POINT (-91.57717 42.03488),-1.598323,0.733647,66,0.020202


In [ ]:
df.to_csv("proximity.csv")